<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_json('/content/drive/MyDrive/MTGdata/AtomicCards.json')

In [71]:
def replace_card_name(index, text):
  name_parts = index.split(',')  # Split by comma
  possible_matches = [index]

  # Add individual name parts if comma exists
  if len(name_parts) > 1:
    possible_matches.extend([part.strip() for part in name_parts])

  # Add permutation for names with multiple words before comma
  first_part = name_parts[0].strip()  # Get the part before comma
  first_part_words = first_part.split()  # Split into words
  if len(first_part_words) > 1:
      possible_matches.append(first_part_words[0]) # Add the first word as a match

  # Replace occurrences of possible matches in the text, using word boundaries
  for name in possible_matches:
    text = re.sub(r'\b' + re.escape(name) + r'\b', 'this', text) # Use re.escape and word boundaries
  return text

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'\/', ' ',text)
  text= re.sub(r'\{|\}', ' ', '',text)
  text = re.sub(r'[^\w\s\+\-\.\:]','',text)
  text = re.sub(r'\n', '.', text)  # Replace new lines with periods
  text = re.sub(r'\.\.+', '.', text)  # Replace multiple periods with one
  return text

In [ ]:
for index, row in df.iterrows():
  try:
    text = row['data'][0]['text']
    text = replace_card_name(index, text)
    text = clean_text(text)
    print(text)
    print('__________________________________________________________________')
  except:
    None